## **Implementación en Algorand**

### **Importar las bibliotecas y los datos**

Para comenzar se instalan los requerimientos

In [27]:
!pip install python-dotenv
!pip install py-algorand-sdk

Ahora se importan todas las librerias a utilizar

In [28]:
import os
import json
import glob
import pandas as pd                                  # Para la manipulación y análisis de datos
import numpy as np                                   # Para crear vectores y matrices n dimensionales
import matplotlib.pyplot as plt                      # Para la generación de gráficas a partir de los datos
import seaborn as sns                                # Para la visualización de datos basado en matplotlib
%matplotlib inline   
import plotly.express as px
import warnings
from dotenv import load_dotenv                      # Para manejar tus variables de entorno
from algosdk.v2client.indexer import IndexerClient  # Para acceder al indexer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
import sklearn.metrics as metrics
pd.set_option('display.max_columns', 35)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

#### **Acceso al indexador**

Para comenzar inicializamos el Algorand Indexer Client y obtendremos el estado del indexador.

In [3]:
load_dotenv()
os.environ['API_KEY'] = 'bH0q8p14jl4XShzytvwUL9o5vZENgynR2PfiT5Hp'

SECRET_KEY = os.getenv("API_KEY")

algod_header = {
    'User-Agent': 'Minimal-PyTeal-SDK-Demo/0.1',
    'X-API-Key': SECRET_KEY
}

indexer_address = "https://mainnet-algorand.api.purestake.io/idx2"

algod_indexer = IndexerClient(
    SECRET_KEY,
    indexer_address,
    algod_header
)

print(algod_indexer.health())

{'data': {'migration-required': False, 'read-only-mode': True}, 'db-available': True, 'is-migrating': False, 'message': '26151855', 'round': 26151855, 'version': '2.15.1'}


#### **Consulta de transacciones**

Ahora se consultarán los datos de la stablecoin USDC, que podemos utilizar para predecir el volumen del mercado que mejora significativamente el margen de maniobra para la liquidación en la cadena. La función get_data para consultar el indexador en función del marco de tiempo dado, en particular, la ronda de inicio y cierre. Además, los datos se escriben en un archivo json y se almacenan en la carpeta datos.

In [9]:
def get_data(asset_id, min_round, end_round):

    reference_point = min_round
    stride = 10000
    while True:
        
        if end_round <= reference_point:
            break
        data = algod_indexer.search_transactions(min_round=reference_point, max_round=reference_point+stride, asset_id=asset_id, min_amount=1, limit=10000)

        len_slice = len(data['transactions']) 

        if end_round > reference_point:

            file=json.dumps(data['transactions'])

            with open(f'data/json_data_{asset_id}_{reference_point}-{stride}.json', 'w') as outfile:
                outfile.write(file)

        reference_point+=10000

min_round = 11611000
max_round = 16711000
usdc = 31566704
get_data(usdc, min_round, max_round)

Una vez obtenidos los datos se transforman los datos para realizar una descripción de los datos

In [29]:
json_dir = os.getcwd() +'/data'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)
dfs = []

for file in file_list:
    with open(file) as f:
        json_loads = json.loads(f.read())
        json_data = pd.json_normalize(json_loads)
    dfs.append(json_data)

df_USDC = pd.concat(dfs)

df_USDC

,close-rewards,closing-amount,confirmed-round,fee,first-valid,genesis-hash,id,intra-round-offset,last-valid,lease,receiver-rewards,round-time,sender,sender-rewards,tx-type,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,asset-transfer-transaction.receiver,signature.logicsig.args,signature.logicsig.logic,signature.logicsig.multisig-signature.subsignature,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version,signature.sig,group,genesis-id,note,asset-transfer-transaction.close-to,signature.multisig.subsignature,signature.multisig.threshold,signature.multisig.version
0,0,0,11611122,1000,11611119,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,QAIGEWDW3GY4MAMV5K3IZR4BLFVMAXCRCYYNMOYCYRZ22FGSJILQ,41,11612119,apZ2NEmRFs/6d73LYzh5BfqyF7XSSy4Pi6e2QLN7iHc=,0,1611336718,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,10084,axfer,7162930000,31566704,0,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,[6zAMlShmCsFTk8tED45rsfC/s4xiTZ/RwBmUtnu+ztyb44w5CJo7wCoW96ovo2VGy3yd5WLICVc6KNX611nwAA==],ASADwMyNCATw1oYPJgIgybvMhz1dbV2yZnPHJQgr6skJgfSbeuhDNGTBiu0oQIEgdKWxEDtStQNW3B5ww6xpYlGP2UWAn3r9TsiMXxnoWAExBCIMMRAjEhAxESQSEDEUKBIQMRctKQQQ,"[{'public-key': 'v8w/YDBK3MmuXQg3njxy1K9cgEZIN2UQWDF3W5vp4MY=', 'signature': 'GE1mVtsR/zOuqo8eMRTcaGLTjFJvCIjcGF40IOxMeaB9+rWPWI+/zB3NNtpmDGkIcCk5IrV3/d7UV1WwJJWnCg=='}, {'public-key': 'otp0n4h9tM8yu8LzH90DCoLInD6MBL/1H/Yuvtm/nYc='}, {'public-key': 'bP/DP9SIymrDqgwZFL733vkGOZAETEnTq5TPT/6wga4=', 'signature': 'umBBx8DIIWNrLD7beelo6Ybn06CGXRTDGZfzT/GLzMEUxD0DA8Uhjc/Z7nGtLmCPhqDe5hW9jRetq/Nah27bAQ=='}]",2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,11611183,1000,11611180,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,6WVY2VA54343UB3VARUWU2PVKIWDVTWS2DOJD2J6KW7GIUVTAJXQ,86,11612180,6DbM9NaiWbclZAjY4y324ZPGzRhWjRTPaAdPO1c45ck=,0,1611336983,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,4108,axfer,179013120000,31566704,0,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,NaN,NaN,NaN,NaN,NaN,ndl/FV8mzsvMzly9fM7iTzEkxVbRjtgkLtgGkdAkVCRfx6Wda69VdlS6voCiuqTaLWUkjo71bX7ktXmoVUWPCw==,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,11611260,1000,11611257,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,DGA7DWZQ32547K7JPY3G6KRJBSOZPGOBU4PO6GEJEDGMUKRKFZKQ,8,11612257,eLqGSRpfz3NQrhvW6HJq0zgabADWeOgUe5MjRdCqdJw=,0,1611337317,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,5042,axfer,10207660000,31566704,0,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,[aK1OuXtuBSLKVmIfBUaG9soQjRTcmmIDV3bv+0PXQRk95qZxkoeaBO2zz/jxZ5Pa0pjJJSk/3jqUqV04deZaDQ==],ASADwMyNCATw1oYPJgIgybvMhz1dbV2yZnPHJQgr6skJgfSbeuhDNGTBiu0oQIEgdKWxEDtStQNW3B5ww6xpYlGP2UWAn3r9TsiMXxnoWAExBCIMMRAjEhAxESQSEDEUKBIQMRctKQQQ,"[{'public-key': 'v8w/YDBK3MmuXQg3njxy1K9cgEZIN2UQWDF3W5vp4MY=', 'signature': 'GE1mVtsR/zOuqo8eMRTcaGLTjFJvCIjcGF40IOxMeaB9+rWPWI+/zB3NNtpmDGkIcCk5IrV3/d7UV1WwJJWnCg=='}, {'public-key': 'otp0n4h9tM8yu8LzH90DCoLInD6MBL/1H/Yuvtm/nYc='}, {'public-key': 'bP/DP9SIymrDqgwZFL733vkGOZAETEnTq5TPT/6wga4=', 'signature': 'umBBx8DIIWNrLD7beelo6Ybn06CGXRTDGZfzT/GLzMEUxD0DA8Uhjc/Z7nGtLmCPhqDe5hW9jRetq/Nah27bAQ=='}]",2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,11611318,1000,11611314,wGHE2Pwdvd7S12BL5FaOP20EGYesN73ktiC1qzkkit8=,BDLQ6PCZ3YMP7MMHMWBWD7MNMMMVOFV4CFLP2CFLGPJOI55VN55Q,12,11612314,QxgUQIckUR9+EjZbdjQQkoURyXCClC7oZundcwQC6EY=,0,1611337569,2UEQTE5QDNXPI7M3TU44G6SYKLFWLPQO7EBZM7K7MHMQQMFI4QJPLHQFHM,5042,axfer,160464270000,31566704,0,ZG54ZBZ5LVWV3MTGOPDSKCBL5LEQTAPUTN5OQQZUMTAYV3JIICA7G3RJZE,[4fUmZJEBpyOLfND9ZNlQ0xlIc3NFRgg4wVduJ6ipkQKdiiZWmIBFIkbHqgis3anl7kHbOHHJQ4SsGPHNsYSNAw==],ASADwMyNCATw1oYPJgIgybvMhz1dbV2yZnPHJQgr6skJgfSbeuhDNGTBiu0oQIEgdKWxEDtStQNW3B5ww6xpYlGP2UWAn3r9TsiMXxnoWAExBCIMMRAjEhAxESQSEDEUKBIQMRctKQQQ,"[{'public-key': 'v8w/YDBK3MmuXQg3njxy1K9cgEZIN2UQWDF3W5vp4MY=', 'signature': 'GE1mVtsR/zOuqo8eMRTcaGLTjFJvCIjcGF40IOxMeaB9+rWPWI+/zB3NNtpmDGkIcCk5IrV3/d7UV1WwJJWnCg=='}, {'public-key': 'otp0n4h9tM8yu8LzH90DC

In [11]:
df_USDC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66855 entries, 0 to 4216
Data columns (total 32 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   close-rewards                                       66855 non-null  int64  
 1   closing-amount                                      66855 non-null  int64  
 2   confirmed-round                                     66855 non-null  int64  
 3   fee                                                 66855 non-null  int64  
 4   first-valid                                         66855 non-null  int64  
 5   genesis-hash                                        66855 non-null  object 
 6   id                                                  66855 non-null  object 
 7   intra-round-offset                                  66855 non-null  int64  
 8   last-valid                                          66855 non-null  int64  
 

In [12]:
df_USDC.describe()

,close-rewards,closing-amount,confirmed-round,fee,first-valid,intra-round-offset,last-valid,receiver-rewards,round-time,sender-rewards,asset-transfer-transaction.amount,asset-transfer-transaction.asset-id,asset-transfer-transaction.close-amount,signature.logicsig.multisig-signature.threshold,signature.logicsig.multisig-signature.version,signature.multisig.threshold,signature.multisig.version
count,66855.0,66855.0,6.685500e+04,66855.000000,6.685500e+04,66855.000000,6.685500e+04,66855.0,6.685500e+04,6.685500e+04,6.685500e+04,66855.0,66855.0,8922.0,8922.0,72.000000,72.0
mean,0.0,0.0,1.448745e+07,1128.017186,1.448745e+07,25.285020,1.448845e+07,0.0,1.623993e+09,3.112819e+06,1.119281e+11,31566704.0,0.0,2.0,1.0,2.527778,1.0
std,0.0,0.0,1.721471e+06,5559.441123,1.721471e+06,32.240918,1.721471e+06,0.0,7.567248e+06,2.645707e+08,9.036957e+11,0.0,0.0,0.0,0.0,0.502731,0.0
min,0.0,0.0,1.161112e+07,1000.000000,1.161112e+07,0.000000,1.161212e+07,0.0,1.611337e+09,0.000000e+00,2.000000e+00,31566704.0,0.0,2.0,1.0,2.000000,1.0
25%,0.0,0.0,1.296338e+07,1000.000000,1.296338e+07,10.000000,1.296438e+07,0.0,1.617285e+09,0.000000e+00,1.648000e+07,31566704.0,0.0,2.0,1.0,2.000000,1.0
50%,0.0,0.0,1.448360e+07,1000.000000,1.448360e+07,20.000000,1.448460e+07,0.0,1.623989e+09,0.000000e+00,3.716100e+08,31566704.0,0.0,2.0,1.0,3.000000,1.0
75%,0.0,0.0,1.624680e+07,1000.000000,1.624680e+07,34.000000,1.624780e+07,0.0,1.631729e+09,3.036000e+03,7.249085e+09,31566704.0,0.0,2.0,1.0,3.000000,1.0
max,0.0,0.0,1.671100e+07,261000.000000,1.671099e+07,2353.000000,1.671199e+07,0.0,1.633758e+09,3.691126e+10,9.700000e+13,31566704.0,0.0,2.0,1.0,3.000000,1.0


Despues se agrupan los datos por día y se suma el monto de la transacción de activos, y se genera una gráfica con estos datos.

In [26]:
df_USDC.fee = df_USDC.fee / 1000000
df_USDC['asset-transfer-transaction.amount'] = df_USDC['asset-transfer-transaction.amount'] / 1000000
df_USDC['round-time'] = pd.to_datetime(df_USDC['round-time'], unit='s')

ag_df = df_USDC.groupby(by=[df_USDC['round-time'].dt.date])['asset-transfer-transaction.amount'].agg(volume='sum', mean='mean')
fig = px.line(ag_df, y=ag_df.volume, x=ag_df.index)
fig.update_layout(template="plotly_dark")
fig.show()

Se agregan características basadas en series de tiempo y se eliminan los datos nulos

In [20]:
ag_df.loc[:,'volume-1'] = ag_df.loc[:,'volume'].shift()

ag_df.loc[:,'volume_diff'] = ag_df.loc[:,'volume'].diff()
ag_df = ag_df.dropna()
ag_df

,volume,mean,volume-1,volume_diff
round-time,,,,
2021-01-24,1.661494e+06,75522.470000,2.036411e+07,-1.870262e+07
2021-01-25,1.348840e+07,85369.613481,1.661494e+06,1.182690e+07
2021-01-26,9.178005e+06,31217.703163,1.348840e+07,-4.310394e+06
2021-01-27,9.007837e+06,38331.220000,9.178005e+06,-1.701680e+05
2021-01-28,2.004708e+07,85671.264957,9.007837e+06,1.103924e+07
...,...,...,...,...
2021-10-05,6.982135e+07,347369.885456,1.433945e+07,5.548189e+07
2021-10-06,2.839528e+07,155165.482715,6.982135e+07,-4.142606e+07
2021-10-07,3.889015e+07,18733.212731,2.839528e+07,1.049487e+07


Se definen conjuntos de datos de prueba y entrenamiento en un 80 %.

In [23]:
train = ag_df[:int(len(ag_df)*0.8)]
test = ag_df[int(len(ag_df)*0.8):]

X_train, X_test, y_train, y_test = train.drop('volume', axis=1), test.drop('volume', axis=1), train['volume'], test['volume']

Se utiliza una red neuronal incorporada con sklearn y el modelo Random Forest. También se hace uso de Gridsearch para comparar y configurar el mejor modelo.

In [24]:
model = RandomForestRegressor(random_state=42)
param_search = { 
    'n_estimators': [10, 20, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(1,15)]
}
tscv = TimeSeriesSplit(n_splits=4)
gsearch = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, scoring = 'neg_mean_squared_error')
gsearch.fit(X_train, y_train)
rf_best_score = gsearch.best_score_
rf_best_model = gsearch.best_estimator_
print(f"{rf_best_model} at {rf_best_score}")


model = MLPRegressor(random_state=1)
param_search = { 
    'max_iter':[ 100, 200, 400, 600, 800, 1000],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
}

tscv = TimeSeriesSplit(n_splits=4)
gsearch = GridSearchCV(estimator=model, cv=tscv, param_grid=param_search, scoring = 'neg_mean_squared_error')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_
print(f"{best_model} at {best_score}")

RandomForestRegressor(max_depth=10, random_state=42) at -121794040854821.55
MLPRegressor(activation='identity', max_iter=100, random_state=1,
             solver='lbfgs') at -0.022252161944998863


#### **Conformación del modelo de pronóstico**

In [21]:
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance, 4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error, 4))
    print('r2: ', round(r2, 4))
    print('MAE: ', round(mean_absolute_error, 4))
    print('MSE: ', round(mse, 4))
    print('RMSE: ', round(np.sqrt(mse), 4))

Por último, se obtiene el rendimiento de los modelos donde la red neuronal supero significativamente al modelo de bosques aleatorios. El resultado de R-cuadrado es perfecto 1 y el error cuadrático medio es cercano a cero.

In [25]:
# Rendimiento de la red neuronal
y_true = y_test.values
y_pred = best_model.predict(X_test)
regression_results(y_true, y_pred)

explained_variance:  1.0
mean_squared_log_error:  0.0
r2:  1.0
MAE:  0.1149
MSE:  0.0199
RMSE:  0.1411


In [21]:
# Rendimiento del modelo de bosques aleatorios
y_true = y_test.values
y_pred = rf_best_model.predict(X_test)
regression_results(y_true, y_pred)

explained_variance:  0.8228
mean_squared_log_error:  0.405
r2:  0.8111
MAE:  7257534.9835
MSE:  319603433267492.56
RMSE:  17877456.0066
